In [93]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from statistics import median
import math

In [2]:
matches = pd.read_csv("/content/sample_data/matches.csv")
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [3]:
# setting up the table with relevant columns
dropList = ["result","player_of_match","venue","umpire1","umpire2","umpire3"]
matches.drop(labels=dropList, axis=1, inplace=True)
matches.head() 

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,Sunrisers Hyderabad,35,0
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,0,Rising Pune Supergiant,0,7
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,0,Kolkata Knight Riders,0,10
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,0,Kings XI Punjab,0,6
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,0,Royal Challengers Bangalore,15,0


In [4]:
matches[pd.isnull(matches['winner'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
300,301,2011,Delhi,2011-05-21,Delhi Daredevils,Pune Warriors,Delhi Daredevils,bat,0,NaN,0,0
545,546,2015,Bangalore,2015-04-29,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,0,NaN,0,0
570,571,2015,Bangalore,2015-05-17,Delhi Daredevils,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,NaN,0,0
744,11340,2019,Bengaluru,30/04/19,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,0,NaN,0,0


In [5]:
matches['winner'].fillna('Draw', inplace=True)
matches[pd.isnull(matches['winner'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets


In [6]:
matches[pd.isnull(matches['city'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
461,462,2014,NaN,2014-04-19,Mumbai Indians,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,Royal Challengers Bangalore,0,7
462,463,2014,NaN,2014-04-19,Kolkata Knight Riders,Delhi Daredevils,Kolkata Knight Riders,bat,0,Delhi Daredevils,0,4
466,467,2014,NaN,2014-04-23,Chennai Super Kings,Rajasthan Royals,Rajasthan Royals,field,0,Chennai Super Kings,7,0
468,469,2014,NaN,2014-04-25,Sunrisers Hyderabad,Delhi Daredevils,Sunrisers Hyderabad,bat,0,Sunrisers Hyderabad,4,0
469,470,2014,NaN,2014-04-25,Mumbai Indians,Chennai Super Kings,Mumbai Indians,bat,0,Chennai Super Kings,0,7
474,475,2014,NaN,2014-04-28,Royal Challengers Bangalore,Kings XI Punjab,Kings XI Punjab,field,0,Kings XI Punjab,0,5
476,477,2014,NaN,2014-04-30,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,0,Sunrisers Hyderabad,15,0


In [7]:
matches['city'].fillna('Dubai', inplace=True)
matches[pd.isnull(matches['city'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets


In [8]:
matches.replace(["Deccan Chargers","Delhi Daredevils"],["Sunrisers Hyderabad","Delhi Capitals"],inplace=True,regex=True)
match1 = matches[((matches.team1=="Mumbai Indians") & (matches.team2=="Chennai Super Kings")) | ((matches.team1=="Chennai Super Kings") & (matches.team2=="Mumbai Indians"))]
match1.shape[0]

28

In [9]:
mw_mi = 0 
mw_csk = 0
lst= [i for i in match1['winner']] 
print("Win Tracker!")
for i in lst:
  if i=='Mumbai Indians':
    mw_mi += 1
  else:
    mw_csk += 1
  print(str(mw_mi)+" "+str(mw_csk))
print("MI vs CSK : "+str(mw_mi)+" "+str(mw_csk))

Win Tracker!
0 1
1 1
2 1
2 2
3 2
3 3
3 4
4 4
5 4
6 4
6 5
7 5
8 5
8 6
9 6
9 7
9 8
9 9
9 10
10 10
11 10
12 10
12 11
13 11
14 11
15 11
16 11
17 11
MI vs CSK : 17 11


In [10]:
last_3_season = matches[(matches['season']>=2017) & np.logical_or(np.logical_and(matches.team1=="Mumbai Indians",matches.team2=="Chennai Super Kings"),np.logical_and(matches.team1=="Chennai Super Kings",matches.team2=="Mumbai Indians"))]
last_3_season.groupby('winner').season.count()

winner
Chennai Super Kings    1
Mumbai Indians         5
Name: season, dtype: int64

Out of 28 matches held between CSK and MI across various stages(qualifier and playoff), MI has majority of 17 wins overall.Same is the situation in past three seasons, MI has steady record of 5 by 1 over CSK.

In [43]:
deliveries = pd.read_csv("/content/sample_data/deliveries.csv")
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,NaN,NaN,NaN


In [44]:
dropToBeList = ['inning','is_super_over','bye_runs','legbye_runs','fielder']
deliveries.drop(dropToBeList, axis=1, inplace=True)
deliveries['dismissal_kind'].fillna('Not Out',inplace=True)
deliveries.head()

,match_id,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,wide_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind
0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
2,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,4,0,4,NaN,Not Out
3,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
4,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,2,0,0,0,2,2,NaN,Not Out


In [45]:
ballbyball = deliveries[((deliveries.batting_team=="Mumbai Indians") & (deliveries.bowling_team=="Chennai Super Kings")) | ((deliveries.batting_team=="Chennai Super Kings") & (deliveries.bowling_team=="Mumbai Indians"))]
ballbyball.head()

,match_id,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,wide_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind
15486,67,Chennai Super Kings,Mumbai Indians,1,1,PA Patel,ML Hayden,SM Pollock,1,0,0,0,1,1,NaN,Not Out
15487,67,Chennai Super Kings,Mumbai Indians,1,2,PA Patel,ML Hayden,SM Pollock,0,0,0,0,0,0,NaN,Not Out
15488,67,Chennai Super Kings,Mumbai Indians,1,3,PA Patel,ML Hayden,SM Pollock,0,0,0,0,0,0,NaN,Not Out
15489,67,Chennai Super Kings,Mumbai Indians,1,4,PA Patel,ML Hayden,SM Pollock,0,0,0,0,0,0,NaN,Not Out
15490,67,Chennai Super Kings,Mumbai Indians,1,5,PA Patel,ML Hayden,SM Pollock,0,0,0,4,0,4,NaN,Not Out


In [46]:
#Q2
dekock = ballbyball[ballbyball['batsman']=='Q de Kock']
matches_played = [i for i in dekock['match_id']]
matches_played = list(set(matches_played))
the_lst = []
for i in matches_played:
  df = dekock[dekock['match_id']==i]
  lst1 = [i for i in df['batsman_runs']]
  lst = [i for i in df['total_runs']]
  count = 0
  if(len(lst)<=10):
    for i in range(len(lst1)):
      if(lst1[i]>=4 and lst1[i]<6):
        count += lst[i]
    if(count>0):
      the_lst.append(count)
  else:
    lst1 = lst1[:10]
    lst = lst[:10]
    for i in range(len(lst1)):
      if(lst1[i]>=4 and lst1[i]<6):
        count += lst[i]
    if(count>0):
      the_lst.append(count)
print(len(matches_played))
print(the_lst)
# in IPL

4
[8, 4, 4]


Against CSK in MI, he played 4 matches and smashed 1 boundary on an average in 10 balls.

In [47]:
#Q2
dekock = deliveries[deliveries['batsman']=='Q de Kock']
matches_played = [i for i in dekock['match_id']]
matches_played = list(set(matches_played))
the_lst = []
for i in matches_played:
  df = dekock[dekock['match_id']==i]
  lst1 = [i for i in df['batsman_runs']]
  lst = [i for i in df['total_runs']]
  count = 0
  if(len(lst)<=10):
    for i in range(len(lst1)):
      if(lst1[i]>=4 and lst1[i]<6):
        count += lst[i]
    if(count>0):
      the_lst.append(count)
  else:
    lst1 = lst1[:10]
    lst = lst[:10]
    for i in range(len(lst1)):
      if(lst1[i]>=4 and lst1[i]<6):
        count += lst[i]
    if(count>0):
      the_lst.append(count)
print(len(matches_played))
print(sum(the_lst)/len(matches_played))
# in IPL

50
4.9


In IPL, in first ten balls he averagely scored 5 runs as boundaries in his matches

In [48]:
#Q4
match_id_lst = list(set([i for i in ballbyball['match_id']]))
#match_id_lst = match_id_lst[-6:]
req_lst=[]
for i in match_id_lst:
  df = ballbyball[(ballbyball['match_id']==i) & (ballbyball['batting_team']=='Chennai Super Kings')]
  df1 = ballbyball[(ballbyball['match_id']==i) & (ballbyball['batting_team']=='Mumbai Indians')]
  over=list(set([k for k in df['over']]))
  over1=list(set([k for k in df1['over']]))
  lst=[]
  count=0
  for j in over:
    data_of_over = df[df['over']==j]
    runs_in_over = [l for l in data_of_over['batsman_runs']]
    if(runs_in_over.count(6) >= 1 and runs_in_over.count(4)>=1):
      count+=1
  for j in over1:
    data_of_over1 = df1[df1['over']==j]
    runs_in_over1 = [l for l in data_of_over1['batsman_runs']]
    if(runs_in_over1.count(6) >= 1 and runs_in_over1.count(4)>=1):
      count+=1
  req_lst.append(count)
print(len(match_id_lst))
print(median(req_lst))

28
4.0


There are four overs where there will be 1 six and 1 four 
in the matches between CSK and MI. That has been improved to five since their last 10 battles!

In [81]:
#Q3
mi_bowling_team = deliveries[deliveries['bowling_team']=='Mumbai Indians']
lst = list(set([i for i in mi_bowling_team['bowler']]))
ratios=[]
for i in lst:
  df = mi_bowling_team[mi_bowling_team['bowler']==i]
  no_of_balls = pd.DataFrame(df.groupby('match_id').ball.count())
  nof_dot_balls = df[(df['total_runs']==0) & (df['extra_runs']==0) & (df['batsman_runs']==0)].groupby('match_id').ball.count()
  val1 = nof_dot_balls.mean()
  val2 = no_of_balls.mean()
  val3 = round((val1/val2),2)
  print(val3.values[0])
  ratios.append(val3.values[0])
  print(i+" "+str(val3))
index_of_maximum_value = ratios.index(max(ratios))
print(lst[index_of_maximum_value])


0.36
JJ Bumrah ball    0.36
dtype: float64
0.43
RD Chahar ball    0.43
dtype: float64
0.33
AM Rahane ball    0.33
dtype: float64
0.32
HH Pandya ball    0.32
dtype: float64
0.38
Z Khan ball    0.38
dtype: float64
nan
STR Binny ball   NaN
dtype: float64
0.37
DJ Bravo ball    0.37
dtype: float64
0.3
RR Raje ball    0.3
dtype: float64
0.33
PJ Sangwan ball    0.33
dtype: float64
0.33
YS Chahal ball    0.33
dtype: float64
0.14
SR Tendulkar ball    0.14
dtype: float64
0.41
AG Murtaza ball    0.41
dtype: float64
0.3
J Suchith ball    0.3
dtype: float64
0.38
MJ McClenaghan ball    0.38
dtype: float64
0.38
SL Malinga ball    0.38
dtype: float64
0.39
A Nel ball    0.39
dtype: float64
0.32
R McLaren ball    0.32
dtype: float64
0.42
MM Patel ball    0.42
dtype: float64
0.42
JDP Oram ball    0.42
dtype: float64
0.42
M de Lange ball    0.42
dtype: float64
0.27
A Roy ball    0.27
dtype: float64
0.28
BB Sran ball    0.28
dtype: float64
0.17
GJ Maxwell ball    0.17
dtype: float64
0.44
NM Coulter-Nile ba

In [83]:
print(ratios[lst.index('JJ Bumrah')])
print(ratios[lst.index('RD Chahar')])
#print(ratios[lst.index('TA Boult')])

0.36
0.43


In [98]:
#Q5
bls_mi=[]
for i in lst:
  df = mi_bowling_team[mi_bowling_team['bowler']==i]
  nof_wickets = df.groupby('match_id').player_dismissed.count()
  no_of_balls = df.groupby('match_id').ball.count()
  nof_dot_balls = df[(df['total_runs']==0) & (df['extra_runs']==0) & (df['batsman_runs']==0)].groupby('match_id').ball.count()
  nof_boundaries = df[df['batsman_runs']==4].groupby('match_id').batsman_runs.count()
  val1 = nof_wickets.mean()+nof_dot_balls.mean()
  val2 = ((nof_boundaries.mean())*4)/(no_of_balls.mean())
  print(i)
  try:
    bls_mi.append(math.ceil(val1-val2))
    print(math.ceil(val1-val2))
  except:
    bls_mi.append(0)
    print(0)
  



JJ Bumrah
10
RD Chahar
11
AM Rahane
0
HH Pandya
6
Z Khan
9
STR Binny
0
DJ Bravo
8
RR Raje
5
PJ Sangwan
3
YS Chahal
8
SR Tendulkar
1
AG Murtaza
11
J Suchith
7
MJ McClenaghan
10
SL Malinga
11
A Nel
8
R McLaren
7
MM Patel
11
JDP Oram
11
M de Lange
12
A Roy
0
BB Sran
3
GJ Maxwell
1
NM Coulter-Nile
12
LMP Simmons
3
Mustafizur Rahman
9
C Nanda
5
RA Shaikh
0
TG Southee
9
JEC Franklin
4
SM Pollock
11
P Kumar
12
BCJ Cutting
4
KV Sharma
10
S Gopal
7
SD Chitnis
6
RW Price
1
NLTC Perera
2
JP Duminy
7
RJ Peterson
6
J Yadav
9
M Markande
7
GR Napier
9
ST Jayasuriya
5
AN Ahmed
8
KA Pollard
5
R Shukla
5
DS Kulkarni
9
N Rana
0
A Joseph
9
DR Smith
6
CJ McKay
7
R Salam
8
PP Ojha
8
KH Pandya
7
R Dhawan
7
J Behrendorff
11
TL Suman
3
MG Johnson
12
R Vinay Kumar
8
P Suyal
8
A Symonds
3
Harbhajan Singh
9
VS Yeligati
3
RG Sharma
2
CJ Anderson
4
AM Nayar
6
CRD Fernando
12
DJ Thornely
5
RP Singh
10
K Santokie
9
MA Khote
3
R Sathish
4
A Nehra
9
A Dananjaya
6


Currently, Bumrah(in the squad) has highest BLS from MI

In [100]:
csk_bowling_team = deliveries[deliveries['bowling_team']=='Chennai Super Kings']
lst1 = list(set([i for i in csk_bowling_team['bowler']]))
for i in lst1:
  df1 = csk_bowling_team[csk_bowling_team['bowler']==i]
  nof_wickets = df1.groupby('match_id').player_dismissed.count()
  no_of_balls = df1.groupby('match_id').ball.count()
  nof_dot_balls = df1[(df1['total_runs']==0) & (df1['extra_runs']==0) & (df1['batsman_runs']==0)].groupby('match_id').ball.count()
  nof_boundaries = df1[df1['batsman_runs']==4].groupby('match_id').batsman_runs.count()
  val1 = nof_wickets.mean()+nof_dot_balls.mean()
  val2 = ((nof_boundaries.mean())*4)/(no_of_balls.mean())
  print(i)
  try:
    bls_mi.append(math.ceil(val1-val2))
    print(math.ceil(val1-val2))
  except:
    bls_mi.append(0)
    print(0)
  



P Amarnath
6
IC Pandey
10
A Flintoff
7
Imran Tahir
10
JW Hastings
9
DJ Bravo
8
DJ Hussey
7
KM Asif
9
P Negi
6
JDP Oram
6
CH Morris
8
CK Kapugedera
0
R Ashwin
10
L Balaji
7
RA Jadeja
7
DE Bollinger
11
JM Kemp
5
BW Hilfenhaus
11
SB Styris
1
DP Nannes
11
AS Rajpoot
4
B Laughlin
5
SB Jakati
8
JA Morkel
9
TG Southee
9
T Thushara
11
KV Sharma
5
SR Watson
6
NLTC Perera
0
S Badree
10
V Shankar
1
L Ngidi
13
DR Smith
4
SN Thakur
9
MM Sharma
10
Joginder Sharma
6
DL Chahar
12
S Randiv
9
SK Raina
5
S Kuggeleijn
10
M Wood
5
JO Holder
8
M Muralitharan
11
C Ganapathy
1
RG More
2
S Tyagi
7
Harbhajan Singh
9
F du Plessis
0
S Vidyut
6
MS Gony
9
M Santner
10
D Willey
7
M Ntini
12
KMDN Kulasekara
10
A Nehra
12
VY Mahesh
7


Jadeja 7, Chahar 12  


---


Pandya 7, Bumrah 10

In [1]:
!pip install jupyter

Using legacy 'setup.py install' for pandocfilters, since package 'wheel' is not installed.
Using legacy 'setup.py install' for pyrsistent, since package 'wheel' is not installed.
    Running setup.py install for pyrsistent: started
    Running setup.py install for pyrsistent: finished with status 'done'
    Running setup.py install for pandocfilters: started
    Running setup.py install for pandocfilters: finished with status 'done'
